# Prologue - Welcome to the dry-run hackathon
Intro slides: https://docs.google.com/presentation/d/12nTDt02UJQ4H0D6iLX40P9MhkuC0h7gXEYzTird-8dw/edit?usp=sharing

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
print(f'Number of cells: {len(df)}')
print(f'Number of columns: {len(df.columns)}')

In [ ]:
# Visualize a cell
import nbvv
from datetime import datetime
from aicsimageprocessing import read_ome_zarr

# Make random seed
seed = int(datetime.now().strftime("%S%f"))

# Indexing to find a cell
a_cell =df[
    (df['gene'] == "TUBA1B") &       #microtubules
    (df['cell_stage'] == "M4M5") &   #"M4/M5" - prometaphase/metaphase
    (df['cell_volume'] > 3000)       #pretty large cell
].sample(1,random_state=seed).iloc[0]

full_img = read_ome_zarr(a_cell["3d_image"])
img_data = full_img.data.squeeze()
print(img_data.shape)
channel_names = full_img.channel_names
print(channel_names)

nbvv.volshow(
    img_data,
    spacing=[1,1,1],  # full_img.physical_pixel_sizes,
    channel_names=channel_names
)